## Real-time Order Book / Top of the Book

You're receiving a stream of limit orders (or quotes) in real time. Each update includes:
- side (BUY or SELL)
- price (integer or float)
- quantity (integer)

You need to maintain data structures that allow you to:
1. Insert an incoming limit order (with price/quantity)
2. Update or Remove existing orders if they are partially/completely filled (in some versions of the puzzle)
3. Query the current best bid (highest BUY price) and best ask (lowest SELL price) in O(log n) or O(1) time

In [ ]:
import heapq

class OrderBook: 
  
  def __init__(self):
    self.buy_heap = []
    self.sell_heap = []
      
  def new_order(self, side, price, quantity):
    if side == "SELL":
      heapq.heappush(self.sell_heap, (price, quantity))
    if side == "BUY":
      heapq.heappush(self.buy_heap, (-price, quantity))
          
  def get_best_bid(self):
    if not self.buy_heap:
      return (None, 0)
    else:
      top = self.sell_heap[0]
      return top
      
ob = OrderBook()
ob.add_order("BUY", 50.5, 100)
ob.add_order("SELL", 51.0, 200)
ob.add_order("BUY", )
  

In [ ]:
# More complete version
import heapq
import itertools

class MatchingOrderBook:
    def __init__(self):
        # Min-heap for sells: store tuples (price, order_id)
        self.sell_heap = []
        # Max-heap for buys: store tuples (-price, order_id)
        self.buy_heap = []

        # Dictionary: order_id -> (price, quantity, side)
        self.orders = {}

        # Generate unique IDs for each new order
        self._id_counter = itertools.count(start=1)

    def add_order(self, side, price, quantity):
        """
        side: 'BUY' or 'SELL'
        price: float
        quantity: int
        """
        if quantity <= 0:
            return None  # invalid

        order_id = next(self._id_counter)
        self.orders[order_id] = (price, quantity, side)

        if side == 'BUY':
            # For a buy order, we add to buy_heap as negative price for max-heap
            heapq.heappush(self.buy_heap, (-price, order_id))
        else:
            # For a sell order
            heapq.heappush(self.sell_heap, (price, order_id))

        # Attempt matching if possible
        if side == 'BUY':
            self._match_buy_order(order_id)
        else:
            self._match_sell_order(order_id)

        return order_id

    def _match_buy_order(self, buy_order_id):
        """
        Try to match the buy order with existing sell orders in the sell_heap.
        Fill as much as possible at the best ask prices <= buy_price.
        """
        if buy_order_id not in self.orders:
            return

        buy_price, buy_qty, side = self.orders[buy_order_id]
        if side != 'BUY':
            return

        # While there's still quantity to buy and we have a sell that can be matched
        while buy_qty > 0 and self.sell_heap:
            best_ask_price, ask_id = self.sell_heap[0]  # top of sell heap
            # Validate the ask order
            if ask_id not in self.orders:
                heapq.heappop(self.sell_heap)  # discard invalid
                continue

            ask_price, ask_qty, ask_side = self.orders[ask_id]
            if ask_side != 'SELL' or ask_qty <= 0:
                # It's inactive, pop and continue
                heapq.heappop(self.sell_heap)
                del self.orders[ask_id]
                continue

            # Check if we can match
            if ask_price <= buy_price:
                # There's a match
                traded_qty = min(buy_qty, ask_qty)
                # Fill
                buy_qty -= traded_qty
                ask_qty -= traded_qty

                # Update the orders
                self.orders[buy_order_id] = (buy_price, buy_qty, side)
                self.orders[ask_id] = (ask_price, ask_qty, ask_side)

                # If the ask is fully filled
                if ask_qty == 0:
                    # remove from active dictionary
                    del self.orders[ask_id]
                    heapq.heappop(self.sell_heap)
                # If the buy is fully filled, we can stop
                if buy_qty == 0:
                    break
            else:
                # No more matching because best ask > buy price
                break

        # If buy_qty == 0 => fully matched
        # else we have partially filled or no fill
        # the updated quantity is already reflected in self.orders.

    def _match_sell_order(self, sell_order_id):
        """
        Symmetric approach for a newly added sell order.
        Match with existing buys in buy_heap if best_bid >= sell_price.
        """
        if sell_order_id not in self.orders:
            return

        sell_price, sell_qty, side = self.orders[sell_order_id]
        if side != 'SELL':
            return

        while sell_qty > 0 and self.buy_heap:
            best_bid_neg_price, bid_id = self.buy_heap[0]
            best_bid_price = -best_bid_neg_price

            if bid_id not in self.orders:
                heapq.heappop(self.buy_heap)
                continue

            bid_price, bid_qty, bid_side = self.orders[bid_id]
            if bid_side != 'BUY' or bid_qty <= 0:
                heapq.heappop(self.buy_heap)
                del self.orders[bid_id]
                continue

            if best_bid_price >= sell_price:
                # Match
                traded_qty = min(sell_qty, bid_qty)
                sell_qty -= traded_qty
                bid_qty -= traded_qty

                self.orders[sell_order_id] = (sell_price, sell_qty, side)
                self.orders[bid_id] = (bid_price, bid_qty, bid_side)

                if bid_qty == 0:
                    del self.orders[bid_id]
                    heapq.heappop(self.buy_heap)
                if sell_qty == 0:
                    break
            else:
                # best bid < sell price, no match
                break

    def get_best_bid(self):
        # Pop invalid or empty orders from the top of buy_heap
        while self.buy_heap:
            neg_price, order_id = self.buy_heap[0]
            if order_id not in self.orders:
                heapq.heappop(self.buy_heap)
                continue
            # check if this order is active
            price, qty, side = self.orders[order_id]
            if side != 'BUY' or qty <= 0:
                heapq.heappop(self.buy_heap)
                del self.orders[order_id]
                continue
            # valid top
            return (price, qty)
        return (None, 0)

    def get_best_ask(self):
        # Pop invalid or empty orders from the top of sell_heap
        while self.sell_heap:
            price, order_id = self.sell_heap[0]
            if order_id not in self.orders:
                heapq.heappop(self.sell_heap)
                continue
            o_price, o_qty, side = self.orders[order_id]
            if side != 'SELL' or o_qty <= 0:
                heapq.heappop(self.sell_heap)
                del self.orders[order_id]
                continue
            # valid top
            return (o_price, o_qty)
        return (None, 0)

# ------------------------------
# Example usage/test
if __name__ == "__main__":
    ob = MatchingOrderBook()

    # Add a SELL at 51.0, quantity 200
    sell_id = ob.add_order("SELL", 51.0, 200)
    print("Best Ask after 1st SELL:", ob.get_best_ask())  # (51.0, 200)

    # Add a BUY at 50.5, quantity 100 => no match, best ask is 51 > 50.5
    buy_id = ob.add_order("BUY", 50.5, 100)
    print("Best Bid:", ob.get_best_bid())    # (50.5, 100)
    print("Best Ask:", ob.get_best_ask())    # (51.0, 200)

    # Add another BUY at 51.2, quantity 150 => should match partially
    # because the best ask is 51.0 which is <= 51.2
    ob.add_order("BUY", 51.2, 150)

    # Let's see what remains after partial fill
    print("Best Bid:", ob.get_best_bid())    # Possibly (50.5, 100) if the 51.2 order was fully filled
    print("Best Ask:", ob.get_best_ask())    # The original SELL might have quantity left or might be fully filled



## Real Time Weighted-Volume Average

You have a **real-time feed of trades**, each containing **(timestamp, price, quantity)**. You need to **maintain and query the volume-weighted average price (VWAP)** for the last \( X \) seconds (or last \( N \) trades).

**Problem Statement**
A typical version:

- **Input:** A stream of **(time, price, size)** arrives in chronological order.
- **At any given time \( t \)**, you want to compute $VWAP_{[t-X, t]} = \frac{\sum_{i \in [t-X, t]} (price_i \times size_i)}{\sum_{i \in [t-X, t]} size_i}$
- **Old trades must be discarded** that are outside the last \( X \) seconds (or the last \( N \) trades, depending on the puzzle).

---

**Key Points**
- **Data Structure:**  
  A **double-ended queue (deque)** often helps for sliding-window problems; or a **balanced tree / queue** with time-based pruning.
  
- **Maintain running sums:**  
  - **Running sum of** \( (price \times quantity) \)  
  - **Running sum of** \( quantity \)  
  - Remove old trades as their timestamps become too old.

- **Complexity Considerations:**
  - Aim for **\( O(1) \)** insertion.
  - **\( O(1) \) or \( O(\log n) \)** removal.
  - Each new query can be answered in **\( O(1) \)** if sums are tracked properly.


In [8]:
from collections import deque
import time

class SlidingWindowVWAP:
    def __init__(self, window_size_seconds):
        self.window_size = window_size_seconds
        self.trades = deque()
        self.cumulative_notional = 0.0
        self.cumulative_volume = 0
        
    def add_trade(self, timestamp, price, quantity):
        trade_value = price * quantity
        self.trades.append((timestamp, trade_value, quantity))
        
        self.cumulative_notional += trade_value
        self.cumulative_volume += quantity
        
        self._prune_old_trades(timestamp)
        
    def get_vwap(self, current_time):
        self._prune_old_trades(current_time)
            
        if self.cumulative_volume == 0:
            return 0.0
        return self.cumulative_notional / self.cumulative_volume
        
    def _prune_old_trades(self, current_time):
        cutoff = current_time - self.window_size
            
        while self.trades and self.trades[0][0] < cutoff:
            old_timestamp, old_value, old_qty = self.trades.popleft()
            self.cumulative_notional -= old_value
            self.cumulative_volume -= old_qty
            
if __name__ == "__main__":
    vwap_calc = SlidingWindowVWAP(10)
    
    now = time.time()
    vwap_calc.add_trade(now - 5, 50.0, 100)
    vwap_calc.add_trade(now - 2, 52.0, 50)
    
    print("Current VWAP:", vwap_calc.get_vwap(now))

Current VWAP: 50.666666666666664



## **Max Profit with (k) Trades / Price Series**

Given an **array** of stock prices over time (e.g., daily closes), find the **maximum profit** you can achieve under certain constraints. Variants:

1. **One transaction**: Buy once, sell once.
2. **Unlimited transactions**: You can buy/sell multiple times, but must sell before buying again.
3. **At most k transactions**: Generalization to limiting how many buy/sell pairs you can perform.

---

## **Basic Version (One Transaction)**
- Standard **"max subarray difference"** problem:
  - Keep track of the **minimum price** encountered so far as you iterate.
  - Compute the **maximum difference** with the current price.

---

## **At Most k Transactions**
- A well-known **dynamic programming approach**:  
  Let `dp[i][j]` be the maximum profit up to day `i` with at most `j` transactions.

- **Recurrence Relation**:

  $dp[i][j] = \max \left( dp[i - 1][j], \max_{m < i} \{ dp[m][j - 1] + (price[i] - price[m]) \} \right)$

- **Optimizations**:  
  - The naive approach takes **\( O(n^2 k) \)** time complexity.
  - Optimized solutions exist to improve efficiency.


## This snippet will be for the **one transaction** variation.

In [3]:
def max_subarray_diff(daily_prices):
  minn = daily_prices[0]
  max_profit = []
  for i, price in enumerate(daily_prices):
    max_profit.append(price - minn)
    minn = min(price, minn)
  return max(max_profit)

print(max_subarray_diff([5,4,13,9,10]))

9


## For the maximum of k transactions

In [ ]:
# dp[i][j] - maximum profit until the i-th day using j transactions
# hold[j] - max profit if you are currently holding a stock on day i with up to j transactions used
# release[j] - maximum profit if you are not holding (i.e. you're in cash) with up to j transactions used



## For unlimited transactions

In [ ]:
# whenever the price is higher tomorrow than today, buy today and sell tomorrow
def max_profit_unlimited(prices):
    if not prices:
        return 0
    
    total_profit = 0
    for i, prices(1, len(prices)):
        if prices[i] > prices[i-1]:
            total_profit += prices[i] - prices[i - 1]
    
    return total_profit

if __name__ = "main":
    prices = [1, 5, 3, 8, 12]
    print("Max Profit (unlimited): ", max_profit_unlimited(prices))